### Graham Kerr
#### graham.s.kerr@NASA.gov; kerrg@cua.edu

<H1 font size="+3" style="color:red">
OZ project Project<br>
-- CX Cross Sections Chebyshev Fits
</H1>

<b>This notebook will fit X-th degree Chebyshev fits to Charge Exchange cross sections from various sources, comparing their relative strengths, and how closely fit results represent either the data or calculations.</b>
 
These functions seem popular in fitting cross section data (e.g. https://amdis.iaea.org/media/db/ORNL/ORNL-Red_Book_-_1.pdf). They fit the Chebyshev polynomials Ti(x) for fitting variable x (see below). Here, as in the ORNL book, I fit ln(sigma) for sigma in cm^2. Note that the conversion to ln(sigma) happens inside the function.<br>
    
    
x = [ (ln(energy)-lin(Emin)) - (ln(Emax) - ln(energy)) ] / (ln(Emax) - ln(Emin))

---
### <b style="color:blue"> Some set up </b>

***Import Modules***

In [ ]:
##
## Import some modules
##

import sys
sys.path.insert(0,'/Users/gskerr1/Documents/Research/Python_Programs/radynpy/')
sys.path.insert(0,'/Users/gskerr1/Documents/Research/OrrallZirkerEffect/')


import radynpy
import OrrallZirkerPy as OZpy
# from OrrallZirkerPy import CrossSections
from OrrallZirkerPy.CrossSections import chebyshev_fn as Cheb


import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, FormatStrFormatter
from matplotlib.ticker import LogLocator
from matplotlib import ticker
import matplotlib.colorbar as cb
import pandas as pd

import cmocean
import colorcet as ccet
import palettable as pal 
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
print(0.38/1e18*1e17)
print(0.38*0.1)
1e-16*1e17

In [ ]:
import importlib
importlib.reload(OZpy.CrossSections)



***Set up plot properties***

In [ ]:
##
## Plot properties (these are just personal preference)
##

#Avenir LT Std
font = {'family': 'Avenir LT Std',
        'color':  'black',
        'weight': 'medium',
        'size': 22,
        }
plot_params = {'ytick.direction': 'in', 
               'xtick.direction': 'in', 
               'xtick.minor.visible': True,
               'ytick.minor.visible': True,
               'xtick.major.size': 10, 'xtick.minor.size': 5,
               'ytick.major.size': 10, 'ytick.minor.size': 5,
               'ytick.right': False,
               'xtick.top': False,
               'ytick.major.width': 1.5,
               'xtick.major.width': 1.5,
               'ytick.minor.width': 1.5,
               'xtick.minor.width': 1.5,
               'axes.linewidth': 1.5,
               'axes.spines.top': False,
               'axes.spines.bottom': True,
               'axes.spines.left': True,
               'axes.spines.right': False,
               'axes.titlepad' : 18 }

plot_lg_params = {'legend.frameon': False}
#plt.rcParams.update({'font.size': font['size'], 'font.family':font['family'], 'font.weight':font['weight'], 'font.color':font['color']})

plt.rcParams.update({'font.size':font['size'], 'font.family':font['family'], 'font.weight':font['weight']})
plt.rcParams.update({'ytick.direction': plot_params['ytick.direction'],
                     'xtick.direction': plot_params['xtick.direction'],
                     'xtick.minor.visible': plot_params['xtick.minor.visible'],
                     'ytick.minor.visible': plot_params['ytick.minor.visible'],
                     'ytick.major.size':  plot_params['ytick.major.size'], 
                     'ytick.minor.size':  plot_params['ytick.minor.size'],
                     'xtick.major.size':  plot_params['xtick.major.size'],                                
                     'xtick.minor.size':  plot_params['xtick.minor.size'],
                     'ytick.right': plot_params['ytick.right'],
                     'xtick.top': plot_params['xtick.top'],
                     'ytick.major.width': plot_params['ytick.major.width'],
                     'xtick.major.width': plot_params['xtick.major.width'],
                     'ytick.minor.width': plot_params['ytick.minor.width'],
                     'xtick.minor.width': plot_params['xtick.minor.width'],                    
                     'axes.linewidth': plot_params['axes.linewidth'],
                     'axes.spines.top' : plot_params['axes.spines.top'],
                     'axes.spines.bottom' : plot_params['axes.spines.bottom'],
                     'axes.spines.left' : plot_params['axes.spines.left'],
                     'axes.spines.right' : plot_params['axes.spines.right'],
                     'axes.titlepad' : plot_params['axes.titlepad'],
                    })

plt.rcParams.update({'legend.frameon': plot_lg_params['legend.frameon']})

mpl.mathtext.SHRINK_FACTOR = 0.6
mpl.mathtext.GROW_FACTOR = 1 / 0.6





template = dict(
        layout = go.Layout(font = dict(family = "Rockwell", size = 18),
                           title_font = dict(family = "Rockwell", size = 22), 
                           plot_bgcolor = 'white',
                           paper_bgcolor = 'white',
                           xaxis = dict(
                                showexponent = 'all',
                                exponentformat = 'e',
                                tickangle = 0,
                                linewidth = 4,
                                showgrid = True,
                            ),
                            yaxis = dict(
                          showexponent = 'all',
                          exponentformat = 'e',
                                linewidth = 4,
                                showgrid = True,
                                anchor = 'free',
                                position = 0,
                                domain = [0.0,1]
                            ),
                            coloraxis_colorbar = dict(
                                thickness = 15,
                                tickformat = '0.2f',
                                ticks = 'outside',
                                titleside = 'right'
                            )
                            ))

---
### <b style="color:blue"> Calculate or extract the cross sections </b>

***Set up the CrossSec object with requested energy values***

In [ ]:
energy = np.arange(1, 8001, 1)
cs = OZpy.CrossSections.CrossSec(energy)

***Run the various functions***

In [ ]:
f95 = cs.cs_fang95()
bw99 = cs.cs_bw99()
kerr = cs.cs_kerr_fit1()
cheshire70 = OZpy.CrossSections.cs_cheshire70()
ludde82 = OZpy.CrossSections.cs_ludde82()
shakeshaft78 = OZpy.CrossSections.cs_shakeshaft78()
bates53 = OZpy.CrossSections.cs_bates53()
winter09 = OZpy.CrossSections.cs_winter09()
belkic92 = OZpy.CrossSections.cs_belkic92()
tselia12 = OZpy.CrossSections.cs_tselia12()

***Combine the data***
> - We are stiching together Winter 2009 (low-E range), and Belkic et al 1992 (high-energy range)
> - From 1-100 keV (winter) and 100-8000 keV (Belkic)

In [ ]:
e1 = 1.00
e2 = 100.00
e3 = 125.00
e4 = 8000.00
eind1 = np.abs(winter09.energy - e1).argmin()
eind2 = np.abs(winter09.energy - e2).argmin()
eind3 = np.abs(belkic92.energy - e3).argmin()
eind4 = np.abs(belkic92.energy - e4).argmin()
energy_tot = np.concatenate((winter09.energy[eind1:eind2+1], belkic92.energy[eind3:eind4+1]))

csec_tot_1= np.concatenate((winter09.Q_p1[eind1:eind2+1],belkic92.Q_p1[eind3:eind4+1]))
csec_tot_2 = np.concatenate((winter09.Q_p2[eind1:eind2+1],belkic92.Q_p2[eind3:eind4+1]))
csec_tot_2s = np.concatenate((winter09.Q_p_2s[eind1:eind2+1],belkic92.Q_p_2s[eind3:eind4+1]))
csec_tot_2p = np.concatenate((winter09.Q_p_2p[eind1:eind2+1],belkic92.Q_p_2p[eind3:eind4+1]))
csec_tot_3 = np.concatenate((winter09.Q_p3[eind1:eind2+1],belkic92.Q_p3[eind3:eind4+1]))
csec_tot_3s = np.concatenate((winter09.Q_p_3s[eind1:eind2+1],belkic92.Q_p_3s[eind3:eind4+1]))
csec_tot_3p = np.concatenate((winter09.Q_p_3p[eind1:eind2+1],belkic92.Q_p_3p[eind3:eind4+1]))
csec_tot_3d = np.concatenate((winter09.Q_p_3d[eind1:eind2+1],belkic92.Q_p_3d[eind3:eind4+1]))

energy2plot = np.arange(1,8001,1)

emin = e1
emax = e4

In [ ]:
len(cfit_n1_4deg)

---
### <b style="color:blue"> Perform Chebyshev fits to data </b>
> - Performs X-th degree Chebyshev fits to the data

***Fit the data***
> - n=1

In [ ]:
cfit_n1_4deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_1, emin = e1, emax = e4, deg = 4)
cfit_n1_5deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_1, emin = e1, emax = e4, deg = 5)
cfit_n1_6deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_1, emin = e1, emax = e4, deg = 6)
cfit_n1_7deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_1, emin = e1, emax = e4, deg = 7)
cfit_n1_8deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_1, emin = e1, emax = e4, deg = 8)

> - n = 2 (total)

In [ ]:
cfit_n2_4deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_2, emin = e1, emax = e4, deg = 4)
cfit_n2_5deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_2, emin = e1, emax = e4, deg = 5)
cfit_n2_6deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_2, emin = e1, emax = e4, deg = 6)
cfit_n2_7deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_2, emin = e1, emax = e4, deg = 7)
cfit_n2_8deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_2, emin = e1, emax = e4, deg = 8)

> - n = 2 (comps)


In [ ]:
cfit_n2s_4deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_2s, emin = e1, emax = e4, deg = 4)
cfit_n2s_5deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_2s, emin = e1, emax = e4, deg = 5)
cfit_n2s_6deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_2s, emin = e1, emax = e4, deg = 6)
cfit_n2s_7deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_2s, emin = e1, emax = e4, deg = 7)
cfit_n2s_8deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_2s, emin = e1, emax = e4, deg = 8)

cfit_n2p_4deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_2p, emin = e1, emax = e4, deg = 4)
cfit_n2p_5deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_2p, emin = e1, emax = e4, deg = 5)
cfit_n2p_6deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_2p, emin = e1, emax = e4, deg = 6)
cfit_n2p_7deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_2p, emin = e1, emax = e4, deg = 7)
cfit_n2p_8deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_2p, emin = e1, emax = e4, deg = 8)

> - n = 3 (total)

In [ ]:
cfit_n3_4deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_3, emin = e1, emax = e4, deg = 4)
cfit_n3_5deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_3, emin = e1, emax = e4, deg = 5)
cfit_n3_6deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_3, emin = e1, emax = e4, deg = 6)
cfit_n3_7deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_3, emin = e1, emax = e4, deg = 7)
cfit_n3_8deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_3, emin = e1, emax = e4, deg = 8)

> - n = 3 (comps)

In [ ]:
cfit_n3s_4deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_3s, emin = e1, emax = e4, deg = 4)
cfit_n3s_5deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_3s, emin = e1, emax = e4, deg = 5)
cfit_n3s_6deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_3s, emin = e1, emax = e4, deg = 6)
cfit_n3s_7deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_3s, emin = e1, emax = e4, deg = 7)
cfit_n3s_8deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_3s, emin = e1, emax = e4, deg = 8)

cfit_n3p_4deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_3p, emin = e1, emax = e4, deg = 4)
cfit_n3p_5deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_3p, emin = e1, emax = e4, deg = 5)
cfit_n3p_6deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_3p, emin = e1, emax = e4, deg = 6)
cfit_n3p_7deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_3p, emin = e1, emax = e4, deg = 7)
cfit_n3p_8deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_3p, emin = e1, emax = e4, deg = 8)

cfit_n3d_4deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_3d, emin = e1, emax = e4, deg = 4)
cfit_n3d_5deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_3d, emin = e1, emax = e4, deg = 5)
cfit_n3d_6deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_3d, emin = e1, emax = e4, deg = 6)
cfit_n3d_7deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_3d, emin = e1, emax = e4, deg = 7)
cfit_n3d_8deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(energy_tot, csec_tot_3d, emin = e1, emax = e4, deg = 8)

---
### <b style="color:blue"> Compare the fits to data </b>




> - n = 1

In [ ]:
savepng = True
fname_out = 'MyFitsCheb_Q_p1_DataComps_extE'
dir1 = './'

logy = True
logx = True

csid = 'H$^{+}$ + H <--> H(1s) + H$^{+}$'
xrange = [1, 8000]
yrange = [1e-9, 500]
cmaptemp = pal.cartocolors.sequential.Sunset_5.get_mpl_colormap()
colors = cmaptemp(np.linspace(0.1,1.0,5))


labels = ['Bates 1953', 'Cheshire 1970', 'Shakeshaft 1978',
          'Ludde et al 1982', 'Winter 2009', 'Belkic et al 1992', 'Tseliakhovich et al 2012',
          '4-deg Cheb', '5-deg Cheb', '6-deg Cheb', '7-deg Cheb', '8-deg Cheb']

if logx == True:
    xtitle = 'E [keV]'
elif logy == False:
    xtitle = 'E [keV]'
if logy == True:
    ytitle = 'Q [10$^{-17}$ cm$^{2}$]'
elif logy == False:
    ytitle = 'Q [10$^{-17}$ cm$^{2}$]'
    
xsize = 8
ysize = 5
fig = plt.figure(figsize=(xsize, ysize))

x1 = 0.075
y1 = 0.075
dx = 0.70
sx = 0.05
dy = 0.70
sy = 0.05

ax1 = fig.add_axes([x1, y1, dx, dy])

ax1.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=True,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=True,
    labeltop = False,
    left=True,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelright=False,
    labelleft = True
    ) 

ax1.spines["left"].set_position(("axes",-0.05))
ax1.spines["bottom"].set_position(("axes",-.075))
ax1.grid(True, axis='x', which='both', linestyle = '-', linewidth = 0.25, color = 'grey', alpha = 0.25)
ax1.grid(True, axis='y', which='both', linestyle = '-', linewidth = 0.25, color = 'grey', alpha = 0.25)

ax1.set_ylim(yrange[0],yrange[-1])
ax1.set_xlim(xrange[0],xrange[-1])
ax1.set_ylabel(ytitle)
ax1.set_xlabel(xtitle)


ax1.text(0.5, 0.90, csid, color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.80, 't = %0.2fs' %(timer[snap]), color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.70, r'$\tau_{exp}$ = %0.2fs' %(exptime), color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.60, '$\lambda_{0}$ = %0.2f$\mathrm{\AA}$'%(lambda_rest), color = 'black',transform=ax1.transAxes,fontsize = 16)     

if logx == True:
    ax1.set_xscale('log')
    ax1.xaxis.set_minor_locator(LogLocator(numticks=15,subs=np.arange(2,10))) #(2)
    for label in ax1.xaxis.get_ticklabels()[::5]:
        label.set_visible(False) #(3)
elif logx == False:
    ax1.set_xscale = 'linear'
if logy == True:
    ax1.set_yscale('log')
    ax1.yaxis.set_minor_locator(LogLocator(numticks=15,subs=np.arange(2,10))) #(2)
    for label in ax1.yaxis.get_ticklabels()[::5]:
        label.set_visible(False) #(3)
elif logy == False:
    ax1.set_yscale = 'linear'
        
line1, = ax1.plot(bates53.energy, bates53.Q_p1,
                   linewidth = 0, linestyle = 'none', color = 'dimgrey',
#                   zorder = 0, alpha = 0.35,
                   marker = 'P', markeredgewidth = 1, 
                   markersize = 8,
                   label = labels[0]
                 )
line2, = ax1.plot(cheshire70.energy, cheshire70.Q_p1,
                   linewidth = 0, linestyle = 'none', color = 'tomato',
#                   zorder = 50, alpha = 1,
                   marker = 'd', markeredgewidth = 1, 
                   markersize = 5,
                   label = labels[1]
                 )
line3, = ax1.plot(shakeshaft78.energy, shakeshaft78.Q_p1,
                   linewidth = 0, linestyle = 'none', color = 'forestgreen',
#                   zorder = 0, alpha = 0.35,
                   marker = 'X', markeredgewidth = 1, 
                   markersize = 5,
                   label = labels[2]
                 )
line4, = ax1.plot(ludde82.energy, ludde82.Q_p1,
                   linewidth = 0, linestyle = 'none', color = 'dodgerblue',
#                   zorder = 0, alpha = 0.35,
                   marker = 'D', markeredgewidth = 1, 
                   markersize = 5,
                   label = labels[3]
                 )
line5, = ax1.plot(winter09.energy, winter09.Q_p1,
                   linewidth = 0, linestyle = 'none', color = 'black',
#                   zorder = 0, alpha = 0.35,
                   marker = 'o', markeredgewidth = 1, 
                   markersize = 8,
                   label = labels[4]
                 )
line6, = ax1.plot(belkic92.energy, belkic92.Q_p1,
                   linewidth = 0, linestyle = 'none', color = 'violet',
#                   zorder = 0, alpha = 0.35,
                   marker = 's', markeredgewidth = 1, 
                   markersize = 8,
                   label = labels[5]
                 )
line7, = ax1.plot(tselia12.energy, tselia12.Q_p1,
                   linewidth = 0, linestyle = 'none', color = 'goldenrod',
#                   zorder = 0, alpha = 0.35,
                   marker = '*', markeredgewidth = 1, 
                   markersize = 5,
                   label = labels[6]
                 )

line1a, = ax1.plot(energy2plot, 
                   (np.exp(Cheb(energy2plot, cfit_n1_4deg, len(cfit_n1_4deg)-1, emin, emax))),
                   linewidth = 4, linestyle = '-', color = colors[0],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[7]
                 )

line1b, = ax1.plot(energy2plot, (np.exp(Cheb(energy2plot, cfit_n1_5deg, len(cfit_n1_5deg)-1, emin, emax))),
                   linewidth = 3, linestyle = '--', color = colors[1],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[8]
                 )
line1c, = ax1.plot(energy2plot, (np.exp(Cheb(energy2plot, cfit_n1_6deg, len(cfit_n1_6deg)-1, emin, emax))),
                   linewidth = 3, linestyle = ':', color = colors[2],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[9]
                 )
line1d, = ax1.plot(energy2plot, (np.exp(Cheb(energy2plot, cfit_n1_7deg, len(cfit_n1_7deg)-1, emin, emax))),
                   linewidth = 3, linestyle = '-.', color = colors[3],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[10]
                 )
line1e, = ax1.plot(energy2plot, (np.exp(Cheb(energy2plot, cfit_n1_8deg, len(cfit_n1_8deg)-1, emin, emax))),
                   linewidth = 2, linestyle = '-', color = colors[4],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[11]
                 )




leg1 = ax1.legend(handles=[line1,line2,line3,line4,line5,line6,line7,
                           line1a, line1b, line1c, line1d, line1e],loc=(1.025,-0.10), fontsize = 15)

# ax1.axvline(x=16, color = 'tomato', alpha = 0.5, linestyle = ':')

if logx == True:
    appendx = '_logx'
elif logx == False:
    appendx = '_linx'
if logy == True:
    appendy = '_logy'
elif logy == False:
    appendy = '_liny'
if savepng == True:
        plt.savefig(dir1+fname_out+appendx+appendy+'.png', format='png', bbox_inches = 'tight', dpi=300)

plt.show()

> - n = 2 (total)

In [ ]:
savepng = True
fname_out = 'MyFitsCheb_Q_p2_DataComps_lowE'
dir1 = './'

logy = True
logx = True

csid = 'H$^{+}$ + H <--> H(n=2) + H$^{+}$'
xrange = [1, 20]
yrange = [1e0, 50]
cmaptemp = pal.cartocolors.sequential.Sunset_5.get_mpl_colormap()
colors = cmaptemp(np.linspace(0.1,1.0,5))


labels = ['Bates 1953', 'Cheshire 1970', 'Shakeshaft 1978',
          'Ludde et al 1982', 'Winter 2009', 'Belkic et al 1992', 'Tseliakhovich et al 2012',
          '4-deg Cheb', '5-deg Cheb', '6-deg Cheb', '7-deg Cheb', '8-deg Cheb']

if logx == True:
    xtitle = 'E [keV]'
elif logy == False:
    xtitle = 'E [keV]'
if logy == True:
    ytitle = 'Q [10$^{-17}$ cm$^{2}$]'
elif logy == False:
    ytitle = 'Q [10$^{-17}$ cm$^{2}$]'
    
xsize = 8
ysize = 5
fig = plt.figure(figsize=(xsize, ysize))

x1 = 0.075
y1 = 0.075
dx = 0.70
sx = 0.05
dy = 0.70
sy = 0.05

ax1 = fig.add_axes([x1, y1, dx, dy])

ax1.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=True,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=True,
    labeltop = False,
    left=True,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelright=False,
    labelleft = True
    ) 

ax1.spines["left"].set_position(("axes",-0.05))
ax1.spines["bottom"].set_position(("axes",-.075))
ax1.grid(True, axis='x', which='both', linestyle = '-', linewidth = 0.25, color = 'grey', alpha = 0.25)
ax1.grid(True, axis='y', which='both', linestyle = '-', linewidth = 0.25, color = 'grey', alpha = 0.25)

ax1.set_ylim(yrange[0],yrange[-1])
ax1.set_xlim(xrange[0],xrange[-1])
ax1.set_ylabel(ytitle)
ax1.set_xlabel(xtitle)


ax1.text(0.5, 0.90, csid, color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.80, 't = %0.2fs' %(timer[snap]), color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.70, r'$\tau_{exp}$ = %0.2fs' %(exptime), color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.60, '$\lambda_{0}$ = %0.2f$\mathrm{\AA}$'%(lambda_rest), color = 'black',transform=ax1.transAxes,fontsize = 16)     

if logx == True:
    ax1.set_xscale('log')
    ax1.xaxis.set_minor_locator(LogLocator(numticks=15,subs=np.arange(2,10))) #(2)
    for label in ax1.xaxis.get_ticklabels()[::5]:
        label.set_visible(False) #(3)
elif logx == False:
    ax1.set_xscale = 'linear'
if logy == True:
    ax1.set_yscale('log')
    ax1.yaxis.set_minor_locator(LogLocator(numticks=15,subs=np.arange(2,10))) #(2)
    for label in ax1.yaxis.get_ticklabels()[::5]:
        label.set_visible(False) #(3)
elif logy == False:
    ax1.set_yscale = 'linear'
        
line1, = ax1.plot(bates53.energy, bates53.Q_p2,
                   linewidth = 0, linestyle = 'none', color = 'dimgrey',
#                   zorder = 0, alpha = 0.35,
                   marker = 'P', markeredgewidth = 1, 
                   markersize = 8,
                   label = labels[0]
                 )
line2, = ax1.plot(cheshire70.energy, cheshire70.Q_p2,
                   linewidth = 0, linestyle = 'none', color = 'tomato',
#                   zorder = 50, alpha = 1,
                   marker = 'd', markeredgewidth = 1, 
                   markersize = 5,
                   label = labels[1]
                 )
line3, = ax1.plot(shakeshaft78.energy, shakeshaft78.Q_p2,
                   linewidth = 0, linestyle = 'none', color = 'forestgreen',
#                   zorder = 0, alpha = 0.35,
                   marker = 'X', markeredgewidth = 1, 
                   markersize = 5,
                   label = labels[2]
                 )
line4, = ax1.plot(ludde82.energy, ludde82.Q_p2,
                   linewidth = 0, linestyle = 'none', color = 'dodgerblue',
#                   zorder = 0, alpha = 0.35,
                   marker = 'D', markeredgewidth = 1, 
                   markersize = 5,
                   label = labels[3]
                 )
line5, = ax1.plot(winter09.energy, winter09.Q_p2,
                   linewidth = 0, linestyle = 'none', color = 'black',
#                   zorder = 0, alpha = 0.35,
                   marker = 'o', markeredgewidth = 1, 
                   markersize = 8,
                   label = labels[4]
                 )
line6, = ax1.plot(belkic92.energy, belkic92.Q_p2,
                   linewidth = 0, linestyle = 'none', color = 'violet',
#                   zorder = 0, alpha = 0.35,
                   marker = 's', markeredgewidth = 1, 
                   markersize = 8,
                   label = labels[5]
                 )
line7, = ax1.plot(tselia12.energy, tselia12.Q_p2,
                   linewidth = 0, linestyle = 'none', color = 'goldenrod',
#                   zorder = 0, alpha = 0.35,
                   marker = '*', markeredgewidth = 1, 
                   markersize = 5,
                   label = labels[6]
                 )

line1a, = ax1.plot(energy2plot, 
                   (np.exp(Cheb(energy2plot, cfit_n2_4deg, len(cfit_n2_4deg)-1, emin, emax))),
                   linewidth = 4, linestyle = '-', color = colors[0],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[7]
                 )

line1b, = ax1.plot(energy2plot, (np.exp(Cheb(energy2plot, cfit_n2_5deg, len(cfit_n2_5deg)-1, emin, emax))),
                   linewidth = 3, linestyle = '--', color = colors[1],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[8]
                 )
line1c, = ax1.plot(energy2plot, (np.exp(Cheb(energy2plot, cfit_n2_6deg, len(cfit_n2_6deg)-1, emin, emax))),
                   linewidth = 3, linestyle = ':', color = colors[2],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[9]
                 )
line1d, = ax1.plot(energy2plot, (np.exp(Cheb(energy2plot, cfit_n2_7deg, len(cfit_n2_7deg)-1, emin, emax))),
                   linewidth = 3, linestyle = '-.', color = colors[3],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[10]
                 )
line1e, = ax1.plot(energy2plot, (np.exp(Cheb(energy2plot, cfit_n2_8deg, len(cfit_n2_8deg)-1, emin, emax))),
                   linewidth = 2, linestyle = '-', color = colors[4],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[11]
                 )




leg1 = ax1.legend(handles=[line1,line2,line3,line4,line5,line6,line7,
                           line1a, line1b, line1c, line1d, line1e],loc=(1.025,-0.10), fontsize = 15)

# ax1.axvline(x=16, color = 'tomato', alpha = 0.5, linestyle = ':')

if logx == True:
    appendx = '_logx'
elif logx == False:
    appendx = '_linx'
if logy == True:
    appendy = '_logy'
elif logy == False:
    appendy = '_liny'
if savepng == True:
        plt.savefig(dir1+fname_out+appendx+appendy+'.png', format='png', bbox_inches = 'tight', dpi=300)

plt.show()

> - n = 2s

In [ ]:
savepng = True
fname_out = 'MyFitsCheb_Q_p2s_DataComps_extE'
dir1 = './'

logy = True
logx = True

csid = 'H$^{+}$ + H <--> H(2s) + H$^{+}$'
xrange = [1, 8000]
yrange = [1e-9, 50]
cmaptemp = pal.cartocolors.sequential.Sunset_5.get_mpl_colormap()
colors = cmaptemp(np.linspace(0.1,1.0,5))


labels = ['Bates 1953', 'Cheshire 1970', 'Shakeshaft 1978',
          'Ludde et al 1982', 'Winter 2009', 'Belkic et al 1992', 'Tseliakhovich et al 2012',
          '4-deg Cheb', '5-deg Cheb', '6-deg Cheb', '7-deg Cheb', '8-deg Cheb']

if logx == True:
    xtitle = 'E [keV]'
elif logy == False:
    xtitle = 'E [keV]'
if logy == True:
    ytitle = 'Q [10$^{-17}$ cm$^{2}$]'
elif logy == False:
    ytitle = 'Q [10$^{-17}$ cm$^{2}$]'
    
xsize = 8
ysize = 5
fig = plt.figure(figsize=(xsize, ysize))

x1 = 0.075
y1 = 0.075
dx = 0.70
sx = 0.05
dy = 0.70
sy = 0.05

ax1 = fig.add_axes([x1, y1, dx, dy])

ax1.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=True,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=True,
    labeltop = False,
    left=True,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelright=False,
    labelleft = True
    ) 

ax1.spines["left"].set_position(("axes",-0.05))
ax1.spines["bottom"].set_position(("axes",-.075))
ax1.grid(True, axis='x', which='both', linestyle = '-', linewidth = 0.25, color = 'grey', alpha = 0.25)
ax1.grid(True, axis='y', which='both', linestyle = '-', linewidth = 0.25, color = 'grey', alpha = 0.25)

ax1.set_ylim(yrange[0],yrange[-1])
ax1.set_xlim(xrange[0],xrange[-1])
ax1.set_ylabel(ytitle)
ax1.set_xlabel(xtitle)


ax1.text(0.5, 0.90, csid, color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.80, 't = %0.2fs' %(timer[snap]), color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.70, r'$\tau_{exp}$ = %0.2fs' %(exptime), color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.60, '$\lambda_{0}$ = %0.2f$\mathrm{\AA}$'%(lambda_rest), color = 'black',transform=ax1.transAxes,fontsize = 16)     

if logx == True:
    ax1.set_xscale('log')
    ax1.xaxis.set_minor_locator(LogLocator(numticks=15,subs=np.arange(2,10))) #(2)
    for label in ax1.xaxis.get_ticklabels()[::5]:
        label.set_visible(False) #(3)
elif logx == False:
    ax1.set_xscale = 'linear'
if logy == True:
    ax1.set_yscale('log')
    ax1.yaxis.set_minor_locator(LogLocator(numticks=15,subs=np.arange(2,10))) #(2)
    for label in ax1.yaxis.get_ticklabels()[::5]:
        label.set_visible(False) #(3)
elif logy == False:
    ax1.set_yscale = 'linear'
        
line1, = ax1.plot(bates53.energy, bates53.Q_p_2s,
                   linewidth = 0, linestyle = 'none', color = 'dimgrey',
#                   zorder = 0, alpha = 0.35,
                   marker = 'P', markeredgewidth = 1, 
                   markersize = 8,
                   label = labels[0]
                 )
line2, = ax1.plot(cheshire70.energy, cheshire70.Q_p_2s,
                   linewidth = 0, linestyle = 'none', color = 'tomato',
#                   zorder = 50, alpha = 1,
                   marker = 'd', markeredgewidth = 1, 
                   markersize = 5,
                   label = labels[1]
                 )
line3, = ax1.plot(shakeshaft78.energy, shakeshaft78.Q_p_2s,
                   linewidth = 0, linestyle = 'none', color = 'forestgreen',
#                   zorder = 0, alpha = 0.35,
                   marker = 'X', markeredgewidth = 1, 
                   markersize = 5,
                   label = labels[2]
                 )
line4, = ax1.plot(ludde82.energy, ludde82.Q_p_2s,
                   linewidth = 0, linestyle = 'none', color = 'dodgerblue',
#                   zorder = 0, alpha = 0.35,
                   marker = 'D', markeredgewidth = 1, 
                   markersize = 5,
                   label = labels[3]
                 )
line5, = ax1.plot(winter09.energy, winter09.Q_p_2s,
                   linewidth = 0, linestyle = 'none', color = 'black',
#                   zorder = 0, alpha = 0.35,
                   marker = 'o', markeredgewidth = 1, 
                   markersize = 8,
                   label = labels[4]
                 )
line6, = ax1.plot(belkic92.energy, belkic92.Q_p_2s,
                   linewidth = 0, linestyle = 'none', color = 'violet',
#                   zorder = 0, alpha = 0.35,
                   marker = 's', markeredgewidth = 1, 
                   markersize = 8,
                   label = labels[5]
                 )
line7, = ax1.plot(tselia12.energy, tselia12.Q_p_2s,
                   linewidth = 0, linestyle = 'none', color = 'goldenrod',
#                   zorder = 0, alpha = 0.35,
                   marker = '*', markeredgewidth = 1, 
                   markersize = 5,
                   label = labels[6]
                 )

line1a, = ax1.plot(energy2plot, 
                   (np.exp(Cheb(energy2plot, cfit_n2s_4deg, len(cfit_n2s_4deg)-1, emin, emax))),
                   linewidth = 4, linestyle = '-', color = colors[0],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[7]
                 )

line1b, = ax1.plot(energy2plot, (np.exp(Cheb(energy2plot, cfit_n2s_5deg, len(cfit_n2s_5deg)-1, emin, emax))),
                   linewidth = 3, linestyle = '--', color = colors[1],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[8]
                 )
line1c, = ax1.plot(energy2plot, (np.exp(Cheb(energy2plot, cfit_n2s_6deg, len(cfit_n2s_6deg)-1, emin, emax))),
                   linewidth = 3, linestyle = ':', color = colors[2],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[9]
                 )
line1d, = ax1.plot(energy2plot, (np.exp(Cheb(energy2plot, cfit_n2s_7deg, len(cfit_n2s_7deg)-1, emin, emax))),
                   linewidth = 3, linestyle = '-.', color = colors[3],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[10]
                 )
line1e, = ax1.plot(energy2plot, (np.exp(Cheb(energy2plot, cfit_n2s_8deg, len(cfit_n2s_8deg)-1, emin, emax))),
                   linewidth = 2, linestyle = '-', color = colors[4],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[11]
                 )




leg1 = ax1.legend(handles=[line1,line2,line3,line4,line5,line6,line7,
                           line1a, line1b, line1c, line1d, line1e],loc=(1.025,-0.10), fontsize = 15)

# ax1.axvline(x=16, color = 'tomato', alpha = 0.5, linestyle = ':')

if logx == True:
    appendx = '_logx'
elif logx == False:
    appendx = '_linx'
if logy == True:
    appendy = '_logy'
elif logy == False:
    appendy = '_liny'
if savepng == True:
        plt.savefig(dir1+fname_out+appendx+appendy+'.png', format='png', bbox_inches = 'tight', dpi=300)

plt.show()

> - n = 2p

In [ ]:
savepng = True
fname_out = 'MyFitsCheb_Q_p2p_DataComps_lowE'
dir1 = './'

logy = True
logx = True

csid = 'H$^{+}$ + H <--> H(2p) + H$^{+}$'
xrange = [1, 20]
yrange = [1e-1, 50]
cmaptemp = pal.cartocolors.sequential.Sunset_5.get_mpl_colormap()
colors = cmaptemp(np.linspace(0.1,1.0,5))


labels = ['Bates 1953', 'Cheshire 1970', 'Shakeshaft 1978',
          'Ludde et al 1982', 'Winter 2009', 'Belkic et al 1992', 'Tseliakhovich et al 2012',
          '4-deg Cheb', '5-deg Cheb', '6-deg Cheb', '7-deg Cheb', '8-deg Cheb']

if logx == True:
    xtitle = 'E [keV]'
elif logy == False:
    xtitle = 'E [keV]'
if logy == True:
    ytitle = 'Q [10$^{-17}$ cm$^{2}$]'
elif logy == False:
    ytitle = 'Q [10$^{-17}$ cm$^{2}$]'
    
xsize = 8
ysize = 5
fig = plt.figure(figsize=(xsize, ysize))

x1 = 0.075
y1 = 0.075
dx = 0.70
sx = 0.05
dy = 0.70
sy = 0.05

ax1 = fig.add_axes([x1, y1, dx, dy])

ax1.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=True,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=True,
    labeltop = False,
    left=True,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelright=False,
    labelleft = True
    ) 

ax1.spines["left"].set_position(("axes",-0.05))
ax1.spines["bottom"].set_position(("axes",-.075))
ax1.grid(True, axis='x', which='both', linestyle = '-', linewidth = 0.25, color = 'grey', alpha = 0.25)
ax1.grid(True, axis='y', which='both', linestyle = '-', linewidth = 0.25, color = 'grey', alpha = 0.25)

ax1.set_ylim(yrange[0],yrange[-1])
ax1.set_xlim(xrange[0],xrange[-1])
ax1.set_ylabel(ytitle)
ax1.set_xlabel(xtitle)


ax1.text(0.5, 0.90, csid, color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.80, 't = %0.2fs' %(timer[snap]), color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.70, r'$\tau_{exp}$ = %0.2fs' %(exptime), color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.60, '$\lambda_{0}$ = %0.2f$\mathrm{\AA}$'%(lambda_rest), color = 'black',transform=ax1.transAxes,fontsize = 16)     

if logx == True:
    ax1.set_xscale('log')
    ax1.xaxis.set_minor_locator(LogLocator(numticks=15,subs=np.arange(2,10))) #(2)
    for label in ax1.xaxis.get_ticklabels()[::5]:
        label.set_visible(False) #(3)
elif logx == False:
    ax1.set_xscale = 'linear'
if logy == True:
    ax1.set_yscale('log')
    ax1.yaxis.set_minor_locator(LogLocator(numticks=15,subs=np.arange(2,10))) #(2)
    for label in ax1.yaxis.get_ticklabels()[::5]:
        label.set_visible(False) #(3)
elif logy == False:
    ax1.set_yscale = 'linear'
        
line1, = ax1.plot(bates53.energy, bates53.Q_p_2p,
                   linewidth = 0, linestyle = 'none', color = 'dimgrey',
#                   zorder = 0, alpha = 0.35,
                   marker = 'P', markeredgewidth = 1, 
                   markersize = 8,
                   label = labels[0]
                 )
line2, = ax1.plot(cheshire70.energy, cheshire70.Q_p_2p,
                   linewidth = 0, linestyle = 'none', color = 'tomato',
#                   zorder = 50, alpha = 1,
                   marker = 'd', markeredgewidth = 1, 
                   markersize = 5,
                   label = labels[1]
                 )
line3, = ax1.plot(shakeshaft78.energy, shakeshaft78.Q_p_2p1+shakeshaft78.Q_p_2p0,
                   linewidth = 0, linestyle = 'none', color = 'forestgreen',
#                   zorder = 0, alpha = 0.35,
                   marker = 'X', markeredgewidth = 1, 
                   markersize = 5,
                   label = labels[2]
                 )
line4, = ax1.plot(ludde82.energy, ludde82.Q_p_2p,
                   linewidth = 0, linestyle = 'none', color = 'dodgerblue',
#                   zorder = 0, alpha = 0.35,
                   marker = 'D', markeredgewidth = 1, 
                   markersize = 5,
                   label = labels[3]
                 )
line5, = ax1.plot(winter09.energy, winter09.Q_p_2p,
                   linewidth = 0, linestyle = 'none', color = 'black',
#                   zorder = 0, alpha = 0.35,
                   marker = 'o', markeredgewidth = 1, 
                   markersize = 8,
                   label = labels[4]
                 )
line6, = ax1.plot(belkic92.energy, belkic92.Q_p_2p,
                   linewidth = 0, linestyle = 'none', color = 'violet',
#                   zorder = 0, alpha = 0.35,
                   marker = 's', markeredgewidth = 1, 
                   markersize = 8,
                   label = labels[5]
                 )
line7, = ax1.plot(tselia12.energy, tselia12.Q_p_2p1+tselia12.Q_p_2p0,
                   linewidth = 0, linestyle = 'none', color = 'goldenrod',
#                   zorder = 0, alpha = 0.35,
                   marker = '*', markeredgewidth = 1, 
                   markersize = 5,
                   label = labels[6]
                 )

line1a, = ax1.plot(energy2plot, 
                   (np.exp(Cheb(energy2plot, cfit_n2p_4deg, len(cfit_n2p_4deg)-1, emin, emax))),
                   linewidth = 4, linestyle = '-', color = colors[0],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[7]
                 )

line1b, = ax1.plot(energy2plot, (np.exp(Cheb(energy2plot, cfit_n2p_5deg, len(cfit_n2p_5deg)-1, emin, emax))),
                   linewidth = 3, linestyle = '--', color = colors[1],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[8]
                 )
line1c, = ax1.plot(energy2plot, (np.exp(Cheb(energy2plot, cfit_n2p_6deg, len(cfit_n2p_6deg)-1, emin, emax))),
                   linewidth = 3, linestyle = ':', color = colors[2],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[9]
                 )
line1d, = ax1.plot(energy2plot, (np.exp(Cheb(energy2plot, cfit_n2p_7deg, len(cfit_n2p_7deg)-1, emin, emax))),
                   linewidth = 3, linestyle = '-.', color = colors[3],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[10]
                 )
line1e, = ax1.plot(energy2plot, (np.exp(Cheb(energy2plot, cfit_n2p_8deg, len(cfit_n2p_8deg)-1, emin, emax))),
                   linewidth = 2, linestyle = '-', color = colors[4],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[11]
                 )




leg1 = ax1.legend(handles=[line1,line2,line3,line4,line5,line6,line7,
                           line1a, line1b, line1c, line1d, line1e],loc=(1.025,-0.10), fontsize = 15)

# ax1.axvline(x=16, color = 'tomato', alpha = 0.5, linestyle = ':')

if logx == True:
    appendx = '_logx'
elif logx == False:
    appendx = '_linx'
if logy == True:
    appendy = '_logy'
elif logy == False:
    appendy = '_liny'
if savepng == True:
        plt.savefig(dir1+fname_out+appendx+appendy+'.png', format='png', bbox_inches = 'tight', dpi=300)

plt.show()

> - n = 3 (total)

In [ ]:
savepng = True
fname_out = 'MyFitsCheb_Q_p3_DataComps_fullE'
dir1 = './'

logy = True
logx = True

csid = 'H$^{+}$ + H <--> H(n=3) + H$^{+}$'
xrange = [1, 8000]
yrange = [1e-9, 5]
cmaptemp = pal.cartocolors.sequential.Sunset_5.get_mpl_colormap()
colors = cmaptemp(np.linspace(0.1,1.0,5))


labels = ['Bates 1953', 'Cheshire 1970', 'Shakeshaft 1978',
          'Ludde et al 1982', 'Winter 2009', 'Belkic et al 1992', 'Tseliakhovich et al 2012',
          '4-deg Cheb', '5-deg Cheb', '6-deg Cheb', '7-deg Cheb', '8-deg Cheb']

if logx == True:
    xtitle = 'E [keV]'
elif logy == False:
    xtitle = 'E [keV]'
if logy == True:
    ytitle = 'Q [10$^{-17}$ cm$^{2}$]'
elif logy == False:
    ytitle = 'Q [10$^{-17}$ cm$^{2}$]'
    
xsize = 8
ysize = 5
fig = plt.figure(figsize=(xsize, ysize))

x1 = 0.075
y1 = 0.075
dx = 0.70
sx = 0.05
dy = 0.70
sy = 0.05

ax1 = fig.add_axes([x1, y1, dx, dy])

ax1.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=True,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=True,
    labeltop = False,
    left=True,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelright=False,
    labelleft = True
    ) 

ax1.spines["left"].set_position(("axes",-0.05))
ax1.spines["bottom"].set_position(("axes",-.075))
ax1.grid(True, axis='x', which='both', linestyle = '-', linewidth = 0.25, color = 'grey', alpha = 0.25)
ax1.grid(True, axis='y', which='both', linestyle = '-', linewidth = 0.25, color = 'grey', alpha = 0.25)

ax1.set_ylim(yrange[0],yrange[-1])
ax1.set_xlim(xrange[0],xrange[-1])
ax1.set_ylabel(ytitle)
ax1.set_xlabel(xtitle)


ax1.text(0.5, 0.90, csid, color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.80, 't = %0.2fs' %(timer[snap]), color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.70, r'$\tau_{exp}$ = %0.2fs' %(exptime), color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.60, '$\lambda_{0}$ = %0.2f$\mathrm{\AA}$'%(lambda_rest), color = 'black',transform=ax1.transAxes,fontsize = 16)     

if logx == True:
    ax1.set_xscale('log')
    ax1.xaxis.set_minor_locator(LogLocator(numticks=15,subs=np.arange(2,10))) #(2)
    for label in ax1.xaxis.get_ticklabels()[::5]:
        label.set_visible(False) #(3)
elif logx == False:
    ax1.set_xscale = 'linear'
if logy == True:
    ax1.set_yscale('log')
    ax1.yaxis.set_minor_locator(LogLocator(numticks=15,subs=np.arange(2,10))) #(2)
    for label in ax1.yaxis.get_ticklabels()[::5]:
        label.set_visible(False) #(3)
elif logy == False:
    ax1.set_yscale = 'linear'
        
line1, = ax1.plot(bates53.energy, bates53.Q_p3,
                   linewidth = 0, linestyle = 'none', color = 'dimgrey',
#                   zorder = 0, alpha = 0.35,
                   marker = 'P', markeredgewidth = 1, 
                   markersize = 8,
                   label = labels[0]
                 )
# line2, = ax1.plot(cheshire70.energy, cheshire70.Q_p3,
#                    linewidth = 0, linestyle = 'none', color = 'tomato',
# #                   zorder = 50, alpha = 1,
#                    marker = 'd', markeredgewidth = 1, 
#                    markersize = 5,
#                    label = labels[1]
#                  )
line3, = ax1.plot(shakeshaft78.energy, shakeshaft78.Q_p3,
                   linewidth = 0, linestyle = 'none', color = 'forestgreen',
#                   zorder = 0, alpha = 0.35,
                   marker = 'X', markeredgewidth = 1, 
                   markersize = 5,
                   label = labels[2]
                 )
line4, = ax1.plot(ludde82.energy, ludde82.Q_p3,
                   linewidth = 0, linestyle = 'none', color = 'dodgerblue',
#                   zorder = 0, alpha = 0.35,
                   marker = 'D', markeredgewidth = 1, 
                   markersize = 5,
                   label = labels[3]
                 )
line5, = ax1.plot(winter09.energy, winter09.Q_p3,
                   linewidth = 0, linestyle = 'none', color = 'black',
#                   zorder = 0, alpha = 0.35,
                   marker = 'o', markeredgewidth = 1, 
                   markersize = 8,
                   label = labels[4]
                 )
line6, = ax1.plot(belkic92.energy, belkic92.Q_p3,
                   linewidth = 0, linestyle = 'none', color = 'violet',
#                   zorder = 0, alpha = 0.35,
                   marker = 's', markeredgewidth = 1, 
                   markersize = 8,
                   label = labels[5]
                 )
line7, = ax1.plot(tselia12.energy, tselia12.Q_p3,
                   linewidth = 0, linestyle = 'none', color = 'goldenrod',
#                   zorder = 0, alpha = 0.35,
                   marker = '*', markeredgewidth = 1, 
                   markersize = 5,
                   label = labels[6]
                 )

line1a, = ax1.plot(energy2plot, 
                   (np.exp(Cheb(energy2plot, cfit_n3_4deg, len(cfit_n3_4deg)-1, emin, emax))),
                   linewidth = 4, linestyle = '-', color = colors[0],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[7]
                 )

line1b, = ax1.plot(energy2plot, (np.exp(Cheb(energy2plot, cfit_n3_5deg, len(cfit_n3_5deg)-1, emin, emax))),
                   linewidth = 3, linestyle = '--', color = colors[1],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[8]
                 )
line1c, = ax1.plot(energy2plot, (np.exp(Cheb(energy2plot, cfit_n3_6deg, len(cfit_n3_6deg)-1, emin, emax))),
                   linewidth = 3, linestyle = ':', color = colors[2],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[9]
                 )
line1d, = ax1.plot(energy2plot, (np.exp(Cheb(energy2plot, cfit_n3_7deg, len(cfit_n3_7deg)-1, emin, emax))),
                   linewidth = 3, linestyle = '-.', color = colors[3],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[10]
                 )
line1e, = ax1.plot(energy2plot, (np.exp(Cheb(energy2plot, cfit_n3_8deg, len(cfit_n3_8deg)-1, emin, emax))),
                   linewidth = 2, linestyle = '-', color = colors[4],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[11]
                 )




leg1 = ax1.legend(handles=[line1,line3,line4,line5,line6,line7,
                           line1a, line1b, line1c, line1d, line1e],loc=(1.025,-0.10), fontsize = 15)

# ax1.axvline(x=16, color = 'tomato', alpha = 0.5, linestyle = ':')

if logx == True:
    appendx = '_logx'
elif logx == False:
    appendx = '_linx'
if logy == True:
    appendy = '_logy'
elif logy == False:
    appendy = '_liny'
if savepng == True:
        plt.savefig(dir1+fname_out+appendx+appendy+'.png', format='png', bbox_inches = 'tight', dpi=300)

plt.show()

> - n = 3s

In [ ]:
savepng = True
fname_out = 'MyFitsCheb_Q_p3s_DataComps_lowE'
dir1 = './'

logy = True
logx = True

csid = 'H$^{+}$ + H <--> H(3s) + H$^{+}$'
xrange = [1, 20]
yrange = [1e-3, 5]
cmaptemp = pal.cartocolors.sequential.Sunset_5.get_mpl_colormap()
colors = cmaptemp(np.linspace(0.1,1.0,5))


labels = ['Bates 1953', 'Cheshire 1970', 'Shakeshaft 1978',
          'Ludde et al 1982', 'Winter 2009', 'Belkic et al 1992', 'Tseliakhovich et al 2012',
          '4-deg Cheb', '5-deg Cheb', '6-deg Cheb', '7-deg Cheb', '8-deg Cheb']

if logx == True:
    xtitle = 'E [keV]'
elif logy == False:
    xtitle = 'E [keV]'
if logy == True:
    ytitle = 'Q [10$^{-17}$ cm$^{2}$]'
elif logy == False:
    ytitle = 'Q [10$^{-17}$ cm$^{2}$]'
    
xsize = 8
ysize = 5
fig = plt.figure(figsize=(xsize, ysize))

x1 = 0.075
y1 = 0.075
dx = 0.70
sx = 0.05
dy = 0.70
sy = 0.05

ax1 = fig.add_axes([x1, y1, dx, dy])

ax1.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=True,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=True,
    labeltop = False,
    left=True,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelright=False,
    labelleft = True
    ) 

ax1.spines["left"].set_position(("axes",-0.05))
ax1.spines["bottom"].set_position(("axes",-.075))
ax1.grid(True, axis='x', which='both', linestyle = '-', linewidth = 0.25, color = 'grey', alpha = 0.25)
ax1.grid(True, axis='y', which='both', linestyle = '-', linewidth = 0.25, color = 'grey', alpha = 0.25)

ax1.set_ylim(yrange[0],yrange[-1])
ax1.set_xlim(xrange[0],xrange[-1])
ax1.set_ylabel(ytitle)
ax1.set_xlabel(xtitle)


ax1.text(0.5, 0.90, csid, color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.80, 't = %0.2fs' %(timer[snap]), color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.70, r'$\tau_{exp}$ = %0.2fs' %(exptime), color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.60, '$\lambda_{0}$ = %0.2f$\mathrm{\AA}$'%(lambda_rest), color = 'black',transform=ax1.transAxes,fontsize = 16)     

if logx == True:
    ax1.set_xscale('log')
    ax1.xaxis.set_minor_locator(LogLocator(numticks=15,subs=np.arange(2,10))) #(2)
    for label in ax1.xaxis.get_ticklabels()[::5]:
        label.set_visible(False) #(3)
elif logx == False:
    ax1.set_xscale = 'linear'
if logy == True:
    ax1.set_yscale('log')
    ax1.yaxis.set_minor_locator(LogLocator(numticks=15,subs=np.arange(2,10))) #(2)
    for label in ax1.yaxis.get_ticklabels()[::5]:
        label.set_visible(False) #(3)
elif logy == False:
    ax1.set_yscale = 'linear'
        
line1, = ax1.plot(bates53.energy, bates53.Q_p_3s,
                   linewidth = 0, linestyle = 'none', color = 'dimgrey',
#                   zorder = 0, alpha = 0.35,
                   marker = 'P', markeredgewidth = 1, 
                   markersize = 8,
                   label = labels[0]
                 )
# line2, = ax1.plot(cheshire70.energy, cheshire70.Q_p_2s,
#                    linewidth = 0, linestyle = 'none', color = 'tomato',
# #                   zorder = 50, alpha = 1,
#                    marker = 'd', markeredgewidth = 1, 
#                    markersize = 5,
#                    label = labels[1]
#                  )
line3, = ax1.plot(shakeshaft78.energy, shakeshaft78.Q_p_3s,
                   linewidth = 0, linestyle = 'none', color = 'forestgreen',
#                   zorder = 0, alpha = 0.35,
                   marker = 'X', markeredgewidth = 1, 
                   markersize = 5,
                   label = labels[2]
                 )
# line4, = ax1.plot(ludde82.energy, ludde82.Q_p_2s,
#                    linewidth = 0, linestyle = 'none', color = 'dodgerblue',
# #                   zorder = 0, alpha = 0.35,
#                    marker = 'D', markeredgewidth = 1, 
#                    markersize = 5,
#                    label = labels[3]
#                  )
line5, = ax1.plot(winter09.energy, winter09.Q_p_3s,
                   linewidth = 0, linestyle = 'none', color = 'black',
#                   zorder = 0, alpha = 0.35,
                   marker = 'o', markeredgewidth = 1, 
                   markersize = 8,
                   label = labels[4]
                 )
line6, = ax1.plot(belkic92.energy, belkic92.Q_p_3s,
                   linewidth = 0, linestyle = 'none', color = 'violet',
#                   zorder = 0, alpha = 0.35,
                   marker = 's', markeredgewidth = 1, 
                   markersize = 8,
                   label = labels[5]
                 )
line7, = ax1.plot(tselia12.energy, tselia12.Q_p_3s,
                   linewidth = 0, linestyle = 'none', color = 'goldenrod',
#                   zorder = 0, alpha = 0.35,
                   marker = '*', markeredgewidth = 1, 
                   markersize = 5,
                   label = labels[6]
                 )

line1a, = ax1.plot(energy2plot, 
                   (np.exp(Cheb(energy2plot, cfit_n3s_4deg, len(cfit_n3s_4deg)-1, emin, emax))),
                   linewidth = 4, linestyle = '-', color = colors[0],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[7]
                 )

line1b, = ax1.plot(energy2plot, (np.exp(Cheb(energy2plot, cfit_n3s_5deg, len(cfit_n3s_5deg)-1, emin, emax))),
                   linewidth = 3, linestyle = '--', color = colors[1],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[8]
                 )
line1c, = ax1.plot(energy2plot, (np.exp(Cheb(energy2plot, cfit_n3s_6deg, len(cfit_n3s_6deg)-1, emin, emax))),
                   linewidth = 3, linestyle = ':', color = colors[2],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[9]
                 )
line1d, = ax1.plot(energy2plot, (np.exp(Cheb(energy2plot, cfit_n3s_7deg, len(cfit_n3s_7deg)-1, emin, emax))),
                   linewidth = 3, linestyle = '-.', color = colors[3],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[10]
                 )
line1e, = ax1.plot(energy2plot, (np.exp(Cheb(energy2plot, cfit_n3s_8deg, len(cfit_n3s_8deg)-1, emin, emax))),
                   linewidth = 2, linestyle = '-', color = colors[4],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[11]
                 )




leg1 = ax1.legend(handles=[line1,line3,line5,line6,line7,
                           line1a, line1b, line1c, line1d, line1e],loc=(1.025,-0.10), fontsize = 15)

# ax1.axvline(x=16, color = 'tomato', alpha = 0.5, linestyle = ':')

if logx == True:
    appendx = '_logx'
elif logx == False:
    appendx = '_linx'
if logy == True:
    appendy = '_logy'
elif logy == False:
    appendy = '_liny'
if savepng == True:
        plt.savefig(dir1+fname_out+appendx+appendy+'.png', format='png', bbox_inches = 'tight', dpi=300)

plt.show()

> - n = 3p

In [ ]:
savepng = True
fname_out = 'MyFitsCheb_Q_pps_DataComps_extE'
dir1 = './'

logy = True
logx = True

csid = 'H$^{+}$ + H <--> H(3p) + H$^{+}$'
xrange = [1, 8000]
yrange = [1e-9, 5]
cmaptemp = pal.cartocolors.sequential.Sunset_5.get_mpl_colormap()
colors = cmaptemp(np.linspace(0.1,1.0,5))


labels = ['Bates 1953', 'Cheshire 1970', 'Shakeshaft 1978',
          'Ludde et al 1982', 'Winter 2009', 'Belkic et al 1992', 'Tseliakhovich et al 2012',
          '4-deg Cheb', '5-deg Cheb', '6-deg Cheb', '7-deg Cheb', '8-deg Cheb']

if logx == True:
    xtitle = 'E [keV]'
elif logy == False:
    xtitle = 'E [keV]'
if logy == True:
    ytitle = 'Q [10$^{-17}$ cm$^{2}$]'
elif logy == False:
    ytitle = 'Q [10$^{-17}$ cm$^{2}$]'
    
xsize = 8
ysize = 5
fig = plt.figure(figsize=(xsize, ysize))

x1 = 0.075
y1 = 0.075
dx = 0.70
sx = 0.05
dy = 0.70
sy = 0.05

ax1 = fig.add_axes([x1, y1, dx, dy])

ax1.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=True,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=True,
    labeltop = False,
    left=True,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelright=False,
    labelleft = True
    ) 

ax1.spines["left"].set_position(("axes",-0.05))
ax1.spines["bottom"].set_position(("axes",-.075))
ax1.grid(True, axis='x', which='both', linestyle = '-', linewidth = 0.25, color = 'grey', alpha = 0.25)
ax1.grid(True, axis='y', which='both', linestyle = '-', linewidth = 0.25, color = 'grey', alpha = 0.25)

ax1.set_ylim(yrange[0],yrange[-1])
ax1.set_xlim(xrange[0],xrange[-1])
ax1.set_ylabel(ytitle)
ax1.set_xlabel(xtitle)


ax1.text(0.5, 0.90, csid, color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.80, 't = %0.2fs' %(timer[snap]), color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.70, r'$\tau_{exp}$ = %0.2fs' %(exptime), color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.60, '$\lambda_{0}$ = %0.2f$\mathrm{\AA}$'%(lambda_rest), color = 'black',transform=ax1.transAxes,fontsize = 16)     

if logx == True:
    ax1.set_xscale('log')
    ax1.xaxis.set_minor_locator(LogLocator(numticks=15,subs=np.arange(2,10))) #(2)
    for label in ax1.xaxis.get_ticklabels()[::5]:
        label.set_visible(False) #(3)
elif logx == False:
    ax1.set_xscale = 'linear'
if logy == True:
    ax1.set_yscale('log')
    ax1.yaxis.set_minor_locator(LogLocator(numticks=15,subs=np.arange(2,10))) #(2)
    for label in ax1.yaxis.get_ticklabels()[::5]:
        label.set_visible(False) #(3)
elif logy == False:
    ax1.set_yscale = 'linear'
        
line1, = ax1.plot(bates53.energy, bates53.Q_p_3p,
                   linewidth = 0, linestyle = 'none', color = 'dimgrey',
#                   zorder = 0, alpha = 0.35,
                   marker = 'P', markeredgewidth = 1, 
                   markersize = 8,
                   label = labels[0]
                 )
# line2, = ax1.plot(cheshire70.energy, cheshire70.Q_p_2s,
#                    linewidth = 0, linestyle = 'none', color = 'tomato',
# #                   zorder = 50, alpha = 1,
#                    marker = 'd', markeredgewidth = 1, 
#                    markersize = 5,
#                    label = labels[1]
#                  )
line3, = ax1.plot(shakeshaft78.energy, shakeshaft78.Q_p_3p0 + shakeshaft78.Q_p_3p1,
                   linewidth = 0, linestyle = 'none', color = 'forestgreen',
#                   zorder = 0, alpha = 0.35,
                   marker = 'X', markeredgewidth = 1, 
                   markersize = 5,
                   label = labels[2]
                 )
# line4, = ax1.plot(ludde82.energy, ludde82.Q_p_2s,
#                    linewidth = 0, linestyle = 'none', color = 'dodgerblue',
# #                   zorder = 0, alpha = 0.35,
#                    marker = 'D', markeredgewidth = 1, 
#                    markersize = 5,
#                    label = labels[3]
#                  )
line5, = ax1.plot(winter09.energy, winter09.Q_p_3p,
                   linewidth = 0, linestyle = 'none', color = 'black',
#                   zorder = 0, alpha = 0.35,
                   marker = 'o', markeredgewidth = 1, 
                   markersize = 8,
                   label = labels[4]
                 )
line6, = ax1.plot(belkic92.energy, belkic92.Q_p_3p,
                   linewidth = 0, linestyle = 'none', color = 'violet',
#                   zorder = 0, alpha = 0.35,
                   marker = 's', markeredgewidth = 1, 
                   markersize = 8,
                   label = labels[5]
                 )
line7, = ax1.plot(tselia12.energy, tselia12.Q_p_3p0 +tselia12.Q_p_3p1 ,
                   linewidth = 0, linestyle = 'none', color = 'goldenrod',
#                   zorder = 0, alpha = 0.35,
                   marker = '*', markeredgewidth = 1, 
                   markersize = 5,
                   label = labels[6]
                 )

line1a, = ax1.plot(energy2plot, 
                   (np.exp(Cheb(energy2plot, cfit_n3p_4deg, len(cfit_n3p_4deg)-1, emin, emax))),
                   linewidth = 4, linestyle = '-', color = colors[0],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[7]
                 )

line1b, = ax1.plot(energy2plot, (np.exp(Cheb(energy2plot, cfit_n3p_5deg, len(cfit_n3p_5deg)-1, emin, emax))),
                   linewidth = 3, linestyle = '--', color = colors[1],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[8]
                 )
line1c, = ax1.plot(energy2plot, (np.exp(Cheb(energy2plot, cfit_n3p_6deg, len(cfit_n3p_6deg)-1, emin, emax))),
                   linewidth = 3, linestyle = ':', color = colors[2],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[9]
                 )
line1d, = ax1.plot(energy2plot, (np.exp(Cheb(energy2plot, cfit_n3p_7deg, len(cfit_n3p_7deg)-1, emin, emax))),
                   linewidth = 3, linestyle = '-.', color = colors[3],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[10]
                 )
line1e, = ax1.plot(energy2plot, (np.exp(Cheb(energy2plot, cfit_n3p_8deg, len(cfit_n3p_8deg)-1, emin, emax))),
                   linewidth = 2, linestyle = '-', color = colors[4],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[11]
                 )




leg1 = ax1.legend(handles=[line1,line3,line5,line6,line7,
                           line1a, line1b, line1c, line1d, line1e],loc=(1.025,-0.10), fontsize = 15)

# ax1.axvline(x=16, color = 'tomato', alpha = 0.5, linestyle = ':')

if logx == True:
    appendx = '_logx'
elif logx == False:
    appendx = '_linx'
if logy == True:
    appendy = '_logy'
elif logy == False:
    appendy = '_liny'
if savepng == True:
        plt.savefig(dir1+fname_out+appendx+appendy+'.png', format='png', bbox_inches = 'tight', dpi=300)

plt.show()

> - n = 3d

In [ ]:
savepng = True
fname_out = 'MyFitsCheb_Q_p3d_DataComps_lowE'
dir1 = './'

logy = True
logx = True

csid = 'H$^{+}$ + H <--> H(3d) + H$^{+}$'
xrange = [1, 20]
yrange = [1e-4, 5]
cmaptemp = pal.cartocolors.sequential.Sunset_5.get_mpl_colormap()
colors = cmaptemp(np.linspace(0.1,1.0,5))


labels = ['Bates 1953', 'Cheshire 1970', 'Shakeshaft 1978',
          'Ludde et al 1982', 'Winter 2009', 'Belkic et al 1992', 'Tseliakhovich et al 2012',
          '4-deg Cheb', '5-deg Cheb', '6-deg Cheb', '7-deg Cheb', '8-deg Cheb']

if logx == True:
    xtitle = 'E [keV]'
elif logy == False:
    xtitle = 'E [keV]'
if logy == True:
    ytitle = 'Q [10$^{-17}$ cm$^{2}$]'
elif logy == False:
    ytitle = 'Q [10$^{-17}$ cm$^{2}$]'
    
xsize = 8
ysize = 5
fig = plt.figure(figsize=(xsize, ysize))

x1 = 0.075
y1 = 0.075
dx = 0.70
sx = 0.05
dy = 0.70
sy = 0.05

ax1 = fig.add_axes([x1, y1, dx, dy])

ax1.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=True,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=True,
    labeltop = False,
    left=True,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelright=False,
    labelleft = True
    ) 

ax1.spines["left"].set_position(("axes",-0.05))
ax1.spines["bottom"].set_position(("axes",-.075))
ax1.grid(True, axis='x', which='both', linestyle = '-', linewidth = 0.25, color = 'grey', alpha = 0.25)
ax1.grid(True, axis='y', which='both', linestyle = '-', linewidth = 0.25, color = 'grey', alpha = 0.25)

ax1.set_ylim(yrange[0],yrange[-1])
ax1.set_xlim(xrange[0],xrange[-1])
ax1.set_ylabel(ytitle)
ax1.set_xlabel(xtitle)


ax1.text(0.5, 0.90, csid, color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.80, 't = %0.2fs' %(timer[snap]), color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.70, r'$\tau_{exp}$ = %0.2fs' %(exptime), color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.60, '$\lambda_{0}$ = %0.2f$\mathrm{\AA}$'%(lambda_rest), color = 'black',transform=ax1.transAxes,fontsize = 16)     

if logx == True:
    ax1.set_xscale('log')
    ax1.xaxis.set_minor_locator(LogLocator(numticks=15,subs=np.arange(2,10))) #(2)
    for label in ax1.xaxis.get_ticklabels()[::5]:
        label.set_visible(False) #(3)
elif logx == False:
    ax1.set_xscale = 'linear'
if logy == True:
    ax1.set_yscale('log')
    ax1.yaxis.set_minor_locator(LogLocator(numticks=15,subs=np.arange(2,10))) #(2)
    for label in ax1.yaxis.get_ticklabels()[::5]:
        label.set_visible(False) #(3)
elif logy == False:
    ax1.set_yscale = 'linear'
        
line1, = ax1.plot(bates53.energy, bates53.Q_p_3d,
                   linewidth = 0, linestyle = 'none', color = 'dimgrey',
#                   zorder = 0, alpha = 0.35,
                   marker = 'P', markeredgewidth = 1, 
                   markersize = 8,
                   label = labels[0]
                 )
# line2, = ax1.plot(cheshire70.energy, cheshire70.Q_p_2s,
#                    linewidth = 0, linestyle = 'none', color = 'tomato',
# #                   zorder = 50, alpha = 1,
#                    marker = 'd', markeredgewidth = 1, 
#                    markersize = 5,
#                    label = labels[1]
#                  )
line3, = ax1.plot(shakeshaft78.energy, shakeshaft78.Q_p_3d0 + shakeshaft78.Q_p_3d1 + shakeshaft78.Q_p_3d2,
                   linewidth = 0, linestyle = 'none', color = 'forestgreen',
#                   zorder = 0, alpha = 0.35,
                   marker = 'X', markeredgewidth = 1, 
                   markersize = 5,
                   label = labels[2]
                 )
# line4, = ax1.plot(ludde82.energy, ludde82.Q_p_2s,
#                    linewidth = 0, linestyle = 'none', color = 'dodgerblue',
# #                   zorder = 0, alpha = 0.35,
#                    marker = 'D', markeredgewidth = 1, 
#                    markersize = 5,
#                    label = labels[3]
#                  )
line5, = ax1.plot(winter09.energy, winter09.Q_p_3d,
                   linewidth = 0, linestyle = 'none', color = 'black',
#                   zorder = 0, alpha = 0.35,
                   marker = 'o', markeredgewidth = 1, 
                   markersize = 8,
                   label = labels[4]
                 )
line6, = ax1.plot(belkic92.energy, belkic92.Q_p_3d,
                   linewidth = 0, linestyle = 'none', color = 'violet',
#                   zorder = 0, alpha = 0.35,
                   marker = 's', markeredgewidth = 1, 
                   markersize = 8,
                   label = labels[5]
                 )
line7, = ax1.plot(tselia12.energy, tselia12.Q_p_3d0 + tselia12.Q_p_3d1 + tselia12.Q_p_3d2 ,
                   linewidth = 0, linestyle = 'none', color = 'goldenrod',
#                   zorder = 0, alpha = 0.35,
                   marker = '*', markeredgewidth = 1, 
                   markersize = 5,
                   label = labels[6]
                 )

line1a, = ax1.plot(energy2plot, 
                   (np.exp(Cheb(energy2plot, cfit_n3d_4deg, len(cfit_n3d_4deg)-1, emin, emax))),
                   linewidth = 4, linestyle = '-', color = colors[0],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[7]
                 )

line1b, = ax1.plot(energy2plot, (np.exp(Cheb(energy2plot, cfit_n3d_5deg, len(cfit_n3d_5deg)-1, emin, emax))),
                   linewidth = 3, linestyle = '--', color = colors[1],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[8]
                 )
line1c, = ax1.plot(energy2plot, (np.exp(Cheb(energy2plot, cfit_n3d_6deg, len(cfit_n3d_6deg)-1, emin, emax))),
                   linewidth = 3, linestyle = ':', color = colors[2],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[9]
                 )
line1d, = ax1.plot(energy2plot, (np.exp(Cheb(energy2plot, cfit_n3d_7deg, len(cfit_n3d_7deg)-1, emin, emax))),
                   linewidth = 3, linestyle = '-.', color = colors[3],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[10]
                 )
line1e, = ax1.plot(energy2plot, (np.exp(Cheb(energy2plot, cfit_n3d_8deg, len(cfit_n3d_8deg)-1, emin, emax))),
                   linewidth = 2, linestyle = '-', color = colors[4],
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[11]
                 )




leg1 = ax1.legend(handles=[line1,line3,line5,line6,line7,
                           line1a, line1b, line1c, line1d, line1e],loc=(1.025,-0.10), fontsize = 15)

# ax1.axvline(x=16, color = 'tomato', alpha = 0.5, linestyle = ':')

if logx == True:
    appendx = '_logx'
elif logx == False:
    appendx = '_linx'
if logy == True:
    appendy = '_logy'
elif logy == False:
    appendy = '_liny'
if savepng == True:
        plt.savefig(dir1+fname_out+appendx+appendy+'.png', format='png', bbox_inches = 'tight', dpi=300)

plt.show()

---
### <b style="color:blue"> Plots the percentage difference between fit and data </b>





In [ ]:
csec_4deg = (np.exp(Cheb(energy_tot, cfit_n1_4deg, len(cfit_n1_4deg)-1, emin, emax)))
csec_5deg = (np.exp(Cheb(energy_tot, cfit_n1_5deg, len(cfit_n1_5deg)-1, emin, emax)))
csec_6deg = (np.exp(Cheb(energy_tot, cfit_n1_6deg, len(cfit_n1_6deg)-1, emin, emax)))
csec_7deg = (np.exp(Cheb(energy_tot, cfit_n1_7deg, len(cfit_n1_7deg)-1, emin, emax)))
csec_8deg = (np.exp(Cheb(energy_tot, cfit_n1_8deg, len(cfit_n1_8deg)-1, emin, emax)))


csec2_4deg = (np.exp(Cheb(energy_tot, cfit_n2_4deg, len(cfit_n2_4deg)-1, emin, emax)))
csec2_5deg = (np.exp(Cheb(energy_tot, cfit_n2_5deg, len(cfit_n2_5deg)-1, emin, emax)))
csec2_6deg = (np.exp(Cheb(energy_tot, cfit_n2_6deg, len(cfit_n2_6deg)-1, emin, emax)))
csec2_7deg = (np.exp(Cheb(energy_tot, cfit_n2_7deg, len(cfit_n2_7deg)-1, emin, emax)))
csec2_8deg = (np.exp(Cheb(energy_tot, cfit_n2_8deg, len(cfit_n2_8deg)-1, emin, emax)))


csec3_4deg = (np.exp(Cheb(energy_tot, cfit_n3_4deg, len(cfit_n3_4deg)-1, emin, emax)))
csec3_5deg = (np.exp(Cheb(energy_tot, cfit_n3_5deg, len(cfit_n3_5deg)-1, emin, emax)))
csec3_6deg = (np.exp(Cheb(energy_tot, cfit_n3_6deg, len(cfit_n3_6deg)-1, emin, emax)))
csec3_7deg = (np.exp(Cheb(energy_tot, cfit_n3_7deg, len(cfit_n3_7deg)-1, emin, emax)))
csec3_8deg = (np.exp(Cheb(energy_tot, cfit_n3_8deg, len(cfit_n3_8deg)-1, emin, emax)))



In [ ]:
savepng = True
fname_out = 'MyFitsCheb_Q_p1_percdiff'
dir1 = './'

logy = False
logx = True

csid = 'H$^{+}$ + H <--> H(1s) + H$^{+}$'
xrange = [1, 5000]
yrange = [-50, 50]
cmaptemp = pal.cartocolors.sequential.Sunset_5.get_mpl_colormap()
colors = cmaptemp(np.linspace(0.1,1.0,5))


labels = ['4-deg poly', '5-deg poly', '6-deg poly', '7-deg poly', '8-deg poly']

if logx == True:
    xtitle = 'E [keV]'
elif logy == False:
    xtitle = 'E [keV]'
if logy == True:
    ytitle = 'Perc Diff'
elif logy == False:
    ytitle = 'Perc Diff'
    
xsize = 8
ysize = 5
fig = plt.figure(figsize=(xsize, ysize))

x1 = 0.075
y1 = 0.075
dx = 0.70
sx = 0.05
dy = 0.70
sy = 0.05

ax1 = fig.add_axes([x1, y1, dx, dy])

ax1.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=True,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=True,
    labeltop = False,
    left=True,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelright=False,
    labelleft = True
    ) 

ax1.spines["left"].set_position(("axes",-0.05))
ax1.spines["bottom"].set_position(("axes",-.075))
ax1.grid(True, axis='x', which='both', linestyle = '-', linewidth = 0.25, color = 'grey', alpha = 0.25)
ax1.grid(True, axis='y', which='both', linestyle = '-', linewidth = 0.25, color = 'grey', alpha = 0.25)

ax1.set_ylim(yrange[0],yrange[-1])
ax1.set_xlim(xrange[0],xrange[-1])
ax1.set_ylabel(ytitle)
ax1.set_xlabel(xtitle)


ax1.text(0.5, 0.90, csid, color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.80, 't = %0.2fs' %(timer[snap]), color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.70, r'$\tau_{exp}$ = %0.2fs' %(exptime), color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.60, '$\lambda_{0}$ = %0.2f$\mathrm{\AA}$'%(lambda_rest), color = 'black',transform=ax1.transAxes,fontsize = 16)     

if logx == True:
    ax1.set_xscale('log')
    ax1.xaxis.set_minor_locator(LogLocator(numticks=15,subs=np.arange(2,10))) #(2)
    for label in ax1.xaxis.get_ticklabels()[::5]:
        label.set_visible(False) #(3)
elif logx == False:
    ax1.set_xscale = 'linear'
if logy == True:
    ax1.set_yscale('log')
    ax1.yaxis.set_minor_locator(LogLocator(numticks=15,subs=np.arange(2,10))) #(2)
    for label in ax1.yaxis.get_ticklabels()[::5]:
        label.set_visible(False) #(3)
elif logy == False:
    ax1.set_yscale = 'linear'
        
line1, = ax1.plot(energy_tot, (csec_4deg - csec_tot_1)/csec_tot_1*100,
                   linewidth = 3, linestyle = '-', color = colors[0],
                  zorder = 0, alpha = 1.0,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[0]
                 )
line2, = ax1.plot(energy_tot, (csec_5deg - csec_tot_1)/csec_tot_1*100,
                   linewidth = 3, linestyle = '-', color = colors[1],
                  zorder = 0, alpha = 1.0,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[1]
                 )
line3, = ax1.plot(energy_tot, (csec_6deg - csec_tot_1)/csec_tot_1*100,
                   linewidth = 3, linestyle = '-', color = colors[2],
                  zorder = 0, alpha = 1.0,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[2]
                 )
line4, = ax1.plot(energy_tot, (csec_7deg - csec_tot_1)/csec_tot_1*100,
                   linewidth = 3, linestyle = '-', color = colors[3],
                  zorder = 0, alpha = 1.0,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[3]
                 )
line5, = ax1.plot(energy_tot, (csec_8deg - csec_tot_1)/csec_tot_1*100,
                   linewidth = 3, linestyle = '-', color = colors[4],
                  zorder = 0, alpha = 1.0,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[4]
                 )



leg1 = ax1.legend(handles=[line1,line2,line3,line4,line5],loc=(1.025,-0.10), fontsize = 15)

# ax1.axvline(x=16, color = 'tomato', alpha = 0.5, linestyle = ':')

if logx == True:
    appendx = '_logx'
elif logx == False:
    appendx = '_linx'
if logy == True:
    appendy = '_logy'
elif logy == False:
    appendy = '_liny'
if savepng == True:
        plt.savefig(dir1+fname_out+appendx+appendy+'.png', format='png', bbox_inches = 'tight', dpi=300)

plt.show()

In [ ]:
savepng = True
fname_out = 'MyFitsCheb_Q_p2_percdiff'
dir1 = './'

logy = False
logx = True

csid = 'H$^{+}$ + H <--> H(n=2) + H$^{+}$'
xrange = [1, 5000]
yrange = [-10, 10]
cmaptemp = pal.cartocolors.sequential.Sunset_5.get_mpl_colormap()
colors = cmaptemp(np.linspace(0.1,1.0,5))


labels = ['4-deg poly', '5-deg poly', '6-deg poly', '7-deg poly', '8-deg poly']

if logx == True:
    xtitle = 'E [keV]'
elif logy == False:
    xtitle = 'E [keV]'
if logy == True:
    ytitle = 'Perc Diff'
elif logy == False:
    ytitle = 'Perc Diff'
    
xsize = 8
ysize = 5
fig = plt.figure(figsize=(xsize, ysize))

x1 = 0.075
y1 = 0.075
dx = 0.70
sx = 0.05
dy = 0.70
sy = 0.05

ax1 = fig.add_axes([x1, y1, dx, dy])

ax1.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=True,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=True,
    labeltop = False,
    left=True,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelright=False,
    labelleft = True
    ) 

ax1.spines["left"].set_position(("axes",-0.05))
ax1.spines["bottom"].set_position(("axes",-.075))
ax1.grid(True, axis='x', which='both', linestyle = '-', linewidth = 0.25, color = 'grey', alpha = 0.25)
ax1.grid(True, axis='y', which='both', linestyle = '-', linewidth = 0.25, color = 'grey', alpha = 0.25)

ax1.set_ylim(yrange[0],yrange[-1])
ax1.set_xlim(xrange[0],xrange[-1])
ax1.set_ylabel(ytitle)
ax1.set_xlabel(xtitle)


ax1.text(0.5, 0.90, csid, color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.80, 't = %0.2fs' %(timer[snap]), color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.70, r'$\tau_{exp}$ = %0.2fs' %(exptime), color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.60, '$\lambda_{0}$ = %0.2f$\mathrm{\AA}$'%(lambda_rest), color = 'black',transform=ax1.transAxes,fontsize = 16)     

if logx == True:
    ax1.set_xscale('log')
    ax1.xaxis.set_minor_locator(LogLocator(numticks=15,subs=np.arange(2,10))) #(2)
    for label in ax1.xaxis.get_ticklabels()[::5]:
        label.set_visible(False) #(3)
elif logx == False:
    ax1.set_xscale = 'linear'
if logy == True:
    ax1.set_yscale('log')
    ax1.yaxis.set_minor_locator(LogLocator(numticks=15,subs=np.arange(2,10))) #(2)
    for label in ax1.yaxis.get_ticklabels()[::5]:
        label.set_visible(False) #(3)
elif logy == False:
    ax1.set_yscale = 'linear'
        
line1, = ax1.plot(energy_tot, (csec2_4deg - csec_tot_2)/csec_tot_1*100,
                   linewidth = 3, linestyle = '-', color = colors[0],
                  zorder = 0, alpha = 1.0,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[0]
                 )
line2, = ax1.plot(energy_tot, (csec2_5deg - csec_tot_2)/csec_tot_1*100,
                   linewidth = 3, linestyle = '-', color = colors[1],
                  zorder = 0, alpha = 1.0,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[1]
                 )
line3, = ax1.plot(energy_tot, (csec2_6deg - csec_tot_2)/csec_tot_1*100,
                   linewidth = 3, linestyle = '-', color = colors[2],
                  zorder = 0, alpha = 1.0,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[2]
                 )
line4, = ax1.plot(energy_tot, (csec2_7deg - csec_tot_2)/csec_tot_1*100,
                   linewidth = 3, linestyle = '-', color = colors[3],
                  zorder = 0, alpha = 1.0,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[3]
                 )
line5, = ax1.plot(energy_tot, (csec2_8deg - csec_tot_2)/csec_tot_1*100,
                   linewidth = 3, linestyle = '-', color = colors[4],
                  zorder = 0, alpha = 1.0,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[4]
                 )



leg1 = ax1.legend(handles=[line1,line2,line3,line4,line5],loc=(1.025,-0.10), fontsize = 15)

# ax1.axvline(x=16, color = 'tomato', alpha = 0.5, linestyle = ':')

if logx == True:
    appendx = '_logx'
elif logx == False:
    appendx = '_linx'
if logy == True:
    appendy = '_logy'
elif logy == False:
    appendy = '_liny'
if savepng == True:
        plt.savefig(dir1+fname_out+appendx+appendy+'.png', format='png', bbox_inches = 'tight', dpi=300)

plt.show()

In [ ]:
savepng = True
fname_out = 'MyFitsCheb_Q_p3_percdiff'
dir1 = './'

logy = False
logx = True

csid = 'H$^{+}$ + H <--> H(n=3) + H$^{+}$'
xrange = [1, 5000]
yrange = [-10, 10]
cmaptemp = pal.cartocolors.sequential.Sunset_5.get_mpl_colormap()
colors = cmaptemp(np.linspace(0.1,1.0,5))


labels = ['4-deg poly', '5-deg poly', '6-deg poly', '7-deg poly', '8-deg poly']

if logx == True:
    xtitle = 'E [keV]'
elif logy == False:
    xtitle = 'E [keV]'
if logy == True:
    ytitle = 'Perc Diff'
elif logy == False:
    ytitle = 'Perc Diff'
    
xsize = 8
ysize = 5
fig = plt.figure(figsize=(xsize, ysize))

x1 = 0.075
y1 = 0.075
dx = 0.70
sx = 0.05
dy = 0.70
sy = 0.05

ax1 = fig.add_axes([x1, y1, dx, dy])

ax1.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=True,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=True,
    labeltop = False,
    left=True,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelright=False,
    labelleft = True
    ) 

ax1.spines["left"].set_position(("axes",-0.05))
ax1.spines["bottom"].set_position(("axes",-.075))
ax1.grid(True, axis='x', which='both', linestyle = '-', linewidth = 0.25, color = 'grey', alpha = 0.25)
ax1.grid(True, axis='y', which='both', linestyle = '-', linewidth = 0.25, color = 'grey', alpha = 0.25)

ax1.set_ylim(yrange[0],yrange[-1])
ax1.set_xlim(xrange[0],xrange[-1])
ax1.set_ylabel(ytitle)
ax1.set_xlabel(xtitle)


ax1.text(0.5, 0.90, csid, color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.80, 't = %0.2fs' %(timer[snap]), color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.70, r'$\tau_{exp}$ = %0.2fs' %(exptime), color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.60, '$\lambda_{0}$ = %0.2f$\mathrm{\AA}$'%(lambda_rest), color = 'black',transform=ax1.transAxes,fontsize = 16)     

if logx == True:
    ax1.set_xscale('log')
    ax1.xaxis.set_minor_locator(LogLocator(numticks=15,subs=np.arange(2,10))) #(2)
    for label in ax1.xaxis.get_ticklabels()[::5]:
        label.set_visible(False) #(3)
elif logx == False:
    ax1.set_xscale = 'linear'
if logy == True:
    ax1.set_yscale('log')
    ax1.yaxis.set_minor_locator(LogLocator(numticks=15,subs=np.arange(2,10))) #(2)
    for label in ax1.yaxis.get_ticklabels()[::5]:
        label.set_visible(False) #(3)
elif logy == False:
    ax1.set_yscale = 'linear'
        
line1, = ax1.plot(energy_tot, (csec3_4deg - csec_tot_3)/csec_tot_1*100,
                   linewidth = 3, linestyle = '-', color = colors[0],
                  zorder = 0, alpha = 1.0,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[0]
                 )
line2, = ax1.plot(energy_tot, (csec3_5deg - csec_tot_3)/csec_tot_1*100,
                   linewidth = 3, linestyle = '-', color = colors[1],
                  zorder = 0, alpha = 1.0,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[1]
                 )
line3, = ax1.plot(energy_tot, (csec3_6deg - csec_tot_3)/csec_tot_1*100,
                   linewidth = 3, linestyle = '-', color = colors[2],
                  zorder = 0, alpha = 1.0,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[2]
                 )
line4, = ax1.plot(energy_tot, (csec3_7deg - csec_tot_3)/csec_tot_1*100,
                   linewidth = 3, linestyle = '-', color = colors[3],
                  zorder = 0, alpha = 1.0,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[3]
                 )
line5, = ax1.plot(energy_tot, (csec3_8deg - csec_tot_3)/csec_tot_1*100,
                   linewidth = 3, linestyle = '-', color = colors[4],
                  zorder = 0, alpha = 1.0,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[4]
                 )



leg1 = ax1.legend(handles=[line1,line2,line3,line4,line5],loc=(1.025,-0.10), fontsize = 15)

# ax1.axvline(x=16, color = 'tomato', alpha = 0.5, linestyle = ':')

if logx == True:
    appendx = '_logx'
elif logx == False:
    appendx = '_linx'
if logy == True:
    appendy = '_logy'
elif logy == False:
    appendy = '_liny'
if savepng == True:
        plt.savefig(dir1+fname_out+appendx+appendy+'.png', format='png', bbox_inches = 'tight', dpi=300)

plt.show()

---
### <b style="color:blue"> Compare my cheb fits to Fang95, BW99, and the poly fits </b>





In [ ]:
energy = np.arange(1, 8001, 1)
cs = OZpy.CrossSections.CrossSec(energy)
f95 = cs.cs_fang95()
bw99 = cs.cs_bw99()
kerr = cs.cs_kerr_poly()
kerr_cheb = cs.cs_kerr_cheb()

In [ ]:
savepng = True
fname_out = 'CompareFits_Kerr_BW99_Fang95_Cheb_Qp1'
dir1 = './'

logy = True
logx = True

csid = 'H$^{+}$ + H <--> H(1s) + H$^{+}$'
xrange = [1, 5000]
yrange = [1e-5, 500]
cmaptemp = pal.cartocolors.sequential.Sunset_5.get_mpl_colormap()
colors = cmaptemp(np.linspace(0.1,1.0,5))


labels = ['Kerr Poly', 'Kerr Cheb', 'Fang95', 'BW99', 'Winter 2009 (1-100 keV)\n+Belkic 1992 (100+ keV)']

if logx == True:
    xtitle = 'E [keV]'
elif logy == False:
    xtitle = 'E [keV]'
if logy == True:
    ytitle = 'Q [10$^{-17}$ cm$^{2}$]'
elif logy == False:
    ytitle = 'Q [10$^{-17}$ cm$^{2}$]'

    
xsize = 8
ysize = 5
fig = plt.figure(figsize=(xsize, ysize))

x1 = 0.075
y1 = 0.075
dx = 0.70
sx = 0.05
dy = 0.70
sy = 0.05

ax1 = fig.add_axes([x1, y1, dx, dy])

ax1.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=True,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=True,
    labeltop = False,
    left=True,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelright=False,
    labelleft = True
    ) 

ax1.spines["left"].set_position(("axes",-0.05))
ax1.spines["bottom"].set_position(("axes",-.075))
ax1.grid(True, axis='x', which='both', linestyle = '-', linewidth = 0.25, color = 'grey', alpha = 0.25)
ax1.grid(True, axis='y', which='both', linestyle = '-', linewidth = 0.25, color = 'grey', alpha = 0.25)

ax1.set_ylim(yrange[0],yrange[-1])
ax1.set_xlim(xrange[0],xrange[-1])
ax1.set_ylabel(ytitle)
ax1.set_xlabel(xtitle)


ax1.text(0.5, 0.90, csid, color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.80, 't = %0.2fs' %(timer[snap]), color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.70, r'$\tau_{exp}$ = %0.2fs' %(exptime), color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.60, '$\lambda_{0}$ = %0.2f$\mathrm{\AA}$'%(lambda_rest), color = 'black',transform=ax1.transAxes,fontsize = 16)     

if logx == True:
    ax1.set_xscale('log')
    ax1.xaxis.set_minor_locator(LogLocator(numticks=15,subs=np.arange(2,10))) #(2)
    for label in ax1.xaxis.get_ticklabels()[::5]:
        label.set_visible(False) #(3)
elif logx == False:
    ax1.set_xscale = 'linear'
if logy == True:
    ax1.set_yscale('log')
    ax1.yaxis.set_minor_locator(LogLocator(numticks=15,subs=np.arange(2,10))) #(2)
    for label in ax1.yaxis.get_ticklabels()[::5]:
        label.set_visible(False) #(3)
elif logy == False:
    ax1.set_yscale = 'linear'
        
line1, = ax1.plot(energy, kerr.Q_p1,
                   linewidth = 4, linestyle = '-', color = 'black',
                  zorder = 0, alpha = 1.0,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[0]
                 )
line2, = ax1.plot(energy, kerr_cheb.Q_p1,
                   linewidth = 4, linestyle = '--', color = 'dimgrey',
                  zorder = 0, alpha = 1.0,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[1]
                 )
line3, = ax1.plot(energy, f95.Q_p1,
                   linewidth = 2, linestyle = '-.', color = 'tomato',
                  zorder = 0, alpha = 1.0,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[2]
                 )
line4, = ax1.plot(energy, bw99.Q_p1,
                   linewidth = 2, linestyle = '--', color = 'dodgerblue',
                  zorder = 0, alpha = 1.0,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[3]
                 )
line5, = ax1.plot(energy_tot, csec_tot_1,
                   linewidth = 0, linestyle = 'none', color = 'goldenrod',
#                   zorder = 0, alpha = 0.35,
                   marker = 'd', markeredgewidth = 1, 
                   markersize = 10,
                   label = labels[4]
                 )


leg1 = ax1.legend(handles=[line1,line2,line3,line4,line5],loc=(1.025,-0.10), fontsize = 15)

# ax1.axvline(x=16, color = 'tomato', alpha = 0.5, linestyle = ':')

if logx == True:
    appendx = '_logx'
elif logx == False:
    appendx = '_linx'
if logy == True:
    appendy = '_logy'
elif logy == False:
    appendy = '_liny'
if savepng == True:
        plt.savefig(dir1+fname_out+appendx+appendy+'.png', format='png', bbox_inches = 'tight', dpi=300)

plt.show()

In [ ]:
savepng = True
fname_out = 'CompareFits_Kerr_BW99_Fang95_Cheb_Qp2'
dir1 = './'

logy = True
logx = True

csid = 'H$^{+}$ + H <--> H(n=2) + H$^{+}$'
xrange = [1, 5000]
yrange = [1e-5, 500]
cmaptemp = pal.cartocolors.sequential.Sunset_5.get_mpl_colormap()
colors = cmaptemp(np.linspace(0.1,1.0,5))


labels = ['Kerr Poly', 'Kerr Cheb', 'Fang95', 'BW99', 'Winter 2009 (1-100 keV)\n+Belkic 1992 (100+ keV)']

if logx == True:
    xtitle = 'E [keV]'
elif logy == False:
    xtitle = 'E [keV]'
if logy == True:
    ytitle = 'Q [10$^{-17}$ cm$^{2}$]'
elif logy == False:
    ytitle = 'Q [10$^{-17}$ cm$^{2}$]'

    
xsize = 8
ysize = 5
fig = plt.figure(figsize=(xsize, ysize))

x1 = 0.075
y1 = 0.075
dx = 0.70
sx = 0.05
dy = 0.70
sy = 0.05

ax1 = fig.add_axes([x1, y1, dx, dy])

ax1.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=True,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=True,
    labeltop = False,
    left=True,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelright=False,
    labelleft = True
    ) 

ax1.spines["left"].set_position(("axes",-0.05))
ax1.spines["bottom"].set_position(("axes",-.075))
ax1.grid(True, axis='x', which='both', linestyle = '-', linewidth = 0.25, color = 'grey', alpha = 0.25)
ax1.grid(True, axis='y', which='both', linestyle = '-', linewidth = 0.25, color = 'grey', alpha = 0.25)

ax1.set_ylim(yrange[0],yrange[-1])
ax1.set_xlim(xrange[0],xrange[-1])
ax1.set_ylabel(ytitle)
ax1.set_xlabel(xtitle)


ax1.text(0.5, 0.90, csid, color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.80, 't = %0.2fs' %(timer[snap]), color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.70, r'$\tau_{exp}$ = %0.2fs' %(exptime), color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.60, '$\lambda_{0}$ = %0.2f$\mathrm{\AA}$'%(lambda_rest), color = 'black',transform=ax1.transAxes,fontsize = 16)     

if logx == True:
    ax1.set_xscale('log')
    ax1.xaxis.set_minor_locator(LogLocator(numticks=15,subs=np.arange(2,10))) #(2)
    for label in ax1.xaxis.get_ticklabels()[::5]:
        label.set_visible(False) #(3)
elif logx == False:
    ax1.set_xscale = 'linear'
if logy == True:
    ax1.set_yscale('log')
    ax1.yaxis.set_minor_locator(LogLocator(numticks=15,subs=np.arange(2,10))) #(2)
    for label in ax1.yaxis.get_ticklabels()[::5]:
        label.set_visible(False) #(3)
elif logy == False:
    ax1.set_yscale = 'linear'
        
line1, = ax1.plot(energy, kerr.Q_p2,
                   linewidth = 4, linestyle = '-', color = 'black',
                  zorder = 0, alpha = 1.0,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[0]
                 )
line2, = ax1.plot(energy, kerr_cheb.Q_p2,
                   linewidth = 4, linestyle = '--', color = 'dimgrey',
                  zorder = 0, alpha = 1.0,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[1]
                 )
line3, = ax1.plot(energy, f95.Q_p2,
                   linewidth = 2, linestyle = '-.', color = 'tomato',
                  zorder = 0, alpha = 1.0,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[2]
                 )
line4, = ax1.plot(energy, bw99.Q_p2,
                   linewidth = 2, linestyle = '--', color = 'dodgerblue',
                  zorder = 0, alpha = 1.0,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[3]
                 )
line5, = ax1.plot(energy_tot, csec_tot_2,
                   linewidth = 0, linestyle = 'none', color = 'goldenrod',
#                   zorder = 0, alpha = 0.35,
                   marker = 'd', markeredgewidth = 1, 
                   markersize = 10,
                   label = labels[4]
                 )


leg1 = ax1.legend(handles=[line1,line2,line3,line4,line5],loc=(1.025,-0.10), fontsize = 15)

# ax1.axvline(x=16, color = 'tomato', alpha = 0.5, linestyle = ':')

if logx == True:
    appendx = '_logx'
elif logx == False:
    appendx = '_linx'
if logy == True:
    appendy = '_logy'
elif logy == False:
    appendy = '_liny'
if savepng == True:
        plt.savefig(dir1+fname_out+appendx+appendy+'.png', format='png', bbox_inches = 'tight', dpi=300)

plt.show()

In [ ]:
savepng = True
fname_out = 'CompareFits_Kerr_BW99_Fang95_Cheb_Qp3'
dir1 = './'

logy = True
logx = True

csid = 'H$^{+}$ + H <--> H(n=3) + H$^{+}$'
xrange = [1, 5000]
yrange = [1e-5, 500]
cmaptemp = pal.cartocolors.sequential.Sunset_5.get_mpl_colormap()
colors = cmaptemp(np.linspace(0.1,1.0,5))


labels = ['Kerr Poly', 'Kerr Cheb', 'Fang95', 'BW99', 'Winter 2009 (1-100 keV)\n+Belkic 1992 (100+ keV)']

if logx == True:
    xtitle = 'E [keV]'
elif logy == False:
    xtitle = 'E [keV]'
if logy == True:
    ytitle = 'Q [10$^{-17}$ cm$^{2}$]'
elif logy == False:
    ytitle = 'Q [10$^{-17}$ cm$^{2}$]'

    
xsize = 8
ysize = 5
fig = plt.figure(figsize=(xsize, ysize))

x1 = 0.075
y1 = 0.075
dx = 0.70
sx = 0.05
dy = 0.70
sy = 0.05

ax1 = fig.add_axes([x1, y1, dx, dy])

ax1.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=True,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=True,
    labeltop = False,
    left=True,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelright=False,
    labelleft = True
    ) 

ax1.spines["left"].set_position(("axes",-0.05))
ax1.spines["bottom"].set_position(("axes",-.075))
ax1.grid(True, axis='x', which='both', linestyle = '-', linewidth = 0.25, color = 'grey', alpha = 0.25)
ax1.grid(True, axis='y', which='both', linestyle = '-', linewidth = 0.25, color = 'grey', alpha = 0.25)

ax1.set_ylim(yrange[0],yrange[-1])
ax1.set_xlim(xrange[0],xrange[-1])
ax1.set_ylabel(ytitle)
ax1.set_xlabel(xtitle)


ax1.text(0.5, 0.90, csid, color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.80, 't = %0.2fs' %(timer[snap]), color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.70, r'$\tau_{exp}$ = %0.2fs' %(exptime), color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.60, '$\lambda_{0}$ = %0.2f$\mathrm{\AA}$'%(lambda_rest), color = 'black',transform=ax1.transAxes,fontsize = 16)     

if logx == True:
    ax1.set_xscale('log')
    ax1.xaxis.set_minor_locator(LogLocator(numticks=15,subs=np.arange(2,10))) #(2)
    for label in ax1.xaxis.get_ticklabels()[::5]:
        label.set_visible(False) #(3)
elif logx == False:
    ax1.set_xscale = 'linear'
if logy == True:
    ax1.set_yscale('log')
    ax1.yaxis.set_minor_locator(LogLocator(numticks=15,subs=np.arange(2,10))) #(2)
    for label in ax1.yaxis.get_ticklabels()[::5]:
        label.set_visible(False) #(3)
elif logy == False:
    ax1.set_yscale = 'linear'
        
line1, = ax1.plot(energy, kerr.Q_p3,
                   linewidth = 4, linestyle = '-', color = 'black',
                  zorder = 0, alpha = 1.0,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[0]
                 )
line2, = ax1.plot(energy, kerr_cheb.Q_p3,
                   linewidth = 4, linestyle = '--', color = 'dimgrey',
                  zorder = 0, alpha = 1.0,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[1]
                 )
line3, = ax1.plot(energy, f95.Q_p3,
                   linewidth = 2, linestyle = '-.', color = 'tomato',
                  zorder = 0, alpha = 1.0,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[2]
                 )
# line4, = ax1.plot(energy, bw99.Q_p3,
#                    linewidth = 2, linestyle = '--', color = 'dodgerblue',
#                   zorder = 0, alpha = 1.0,
# #                    marker = 'P', markeredgewidth = 1, 
# #                    markersize = 8,
#                    label = labels[3]
#                  )
line5, = ax1.plot(energy_tot, csec_tot_3,
                   linewidth = 0, linestyle = 'none', color = 'goldenrod',
#                   zorder = 0, alpha = 0.35,
                   marker = 'd', markeredgewidth = 1, 
                   markersize = 10,
                   label = labels[4]
                 )


leg1 = ax1.legend(handles=[line1,line2,line3,line5],loc=(1.025,-0.10), fontsize = 15)

# ax1.axvline(x=16, color = 'tomato', alpha = 0.5, linestyle = ':')

if logx == True:
    appendx = '_logx'
elif logx == False:
    appendx = '_linx'
if logy == True:
    appendy = '_logy'
elif logy == False:
    appendy = '_liny'
if savepng == True:
        plt.savefig(dir1+fname_out+appendx+appendy+'.png', format='png', bbox_inches = 'tight', dpi=300)

plt.show()